In [ ]:
import kwant
import numpy as np
import adaptive
adaptive.notebook_extension()

import matplotlib.pyplot as plt

import scipy.constants
import cmath

import os, sys
sys.path.append(os.path.abspath('../../two_dim_majoranas/'))

import sns_system, spectrum
import holoviews as hv

In [ ]:
def update_coloring(site_colors):
    coloring =  {'middle_interior' : 'grey',
                 'middle_barrier' : 'black',
                 'bottom_superconductor' : 'gold',
                 'top_superconductor' : 'gold',
                 'top_cut' : 'red',
                 'bottom_cut' : 'blue'}

    for k,v in site_colors.items():
        if site_colors.get(k) is not None:
            try:
                site_colors[k] = coloring[v]
            except:
                pass


# Sytem dimension

In [ ]:
syst_pars = dict(
    L_m = 200,
    L_x = 1600,
    L_sc_up = 4000,
    L_sc_down = 4000,
    z_x = 800,
    z_y = 200,
    a   = 10,
    shape = 'parallel_curve',
    transverse_soi = True,
    mu_from_bottom_of_spin_orbit_bands = True,
    k_x_in_sc = True,
    wraparound = False,
    current = False,
    ns_junction=False)

# Make normal zigzag system

In [ ]:
%%time
syst, site_colors, hopping = sns_system.make_system(**syst_pars)
update_coloring(site_colors)

In [ ]:
kwant.plot(syst,site_color=sns_system.site_color_function(site_colors, syst));

# Define parameters

In [ ]:
constants = dict(
    m_eff=0.02 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

params_raw= dict(g_factor_middle = 26,
                 g_factor_left = 0,
                 g_factor_right = 0,
                 mu = 10,
                 alpha_middle = 20,
                 alpha_left = 0,
                 alpha_right = 0,
                 Delta_left = .2,
                 Delta_right = .2,
                 B_x = 1, B_y = 0*-.1, B_z = 0,
                 phase = np.pi,
                 T = 0.0,
                 V = 0)

params = dict(**constants,
              **params_raw)

# Calculate spectrum and display majorana wavefunction

In [ ]:
e, ev = spectrum.calc_spectrum(syst, params, k= 4)
print(e)

In [ ]:
d=kwant.operator.Density(syst)

fig, ax = plt.subplots()
kwant.plot(syst,site_color=sns_system.site_color_function(site_colors, syst), dpi=400, ax=ax)
kwant.plotter.map(syst, d(ev[:,0]), ax=ax)
ax.arrow(1000, 0, 200*params['B_x'],  200*params['B_y'],
        width=40, color='r')

# Create straight wire

In [ ]:
sp=syst_pars.copy()
sp['wraparound'] = True
sp['shape'] = False

sp['L_m'] = 200
sp['L_x'] = sp['a']
syst_straight, site_colors_straight, _ = sns_system.make_system(**sp)
update_coloring(site_colors_straight)

### Adjust magnetic field and calculate maximum k_x

In [ ]:
def get_k_and_alpha(z_x, z_y, L_m, top_of_soi, **kwargs):
    import formulas
    theta = np.arctan(4*z_y/z_x)
    L_y = 2*L_m/np.cos(theta)
    Lp = np.sqrt((z_y + L_y)**2 + (z_x/4)**2)
    
    x = np.sqrt(L_y**2 - L_m**2)
    Ld = np.sqrt((z_x/4)**2 + z_y**2)
    
    kp = 2*np.pi/Lp
    alpha = np.arctan(4*(L_y + z_y)/z_x)-theta
    
    if top_of_soi:
        energy = params['mu'] + params['m_eff']*params['alpha_middle']**2 / (params['hbar']**2)
        kf = formulas.fermi_wavenumber(energy, 0.02) * syst_pars['a']
    else:
        kf = formulas.fermi_wavenumber(params['mu'], 0.02) * syst_pars['a']
    
    return (np.cos(alpha)*kf, alpha, theta)

kp, alpha, theta = get_k_and_alpha(top_of_soi=False, **syst_pars, **params)
print("kp =", kp)

### Calculate adjusted magnetic field and plot its base to prove it is correct

In [ ]:
B = np.sqrt(params['B_x']**2 + params['B_y']**2)

params['B_x'] = B * np.cos(theta)
params['B_y'] = -B * np.sin(theta)

# fig, ax = plt.subplots(dpi=100)
# kwant.plot(syst, site_color=sns_system.site_color_function(site_colors, syst), ax=ax)
# ax.arrow(0, 0, 200*np.cos(theta), 200*np.sin(theta),
#         width=2, color='r')
# ax.set_aspect(1)

### Show adjusted magnetic field

In [ ]:
params

In [ ]:
N = 100
fig, ax = plt.subplots(dpi=40)
kwant.plot(syst_straight, site_color=sns_system.site_color_function(site_colors_straight, syst_straight), ax=ax)
# ax.arrow(0, 0, 100*params['B_x'],  100*params['B_y'],
#         width=10, color='r')
ax.set_aspect(1)
ax.set_xlim(-2*sp['a'], 20*sp['a']);
ylim= sp['a']*(np.array(ax.get_ylim())//sp['a'])
ylim-=ylim%(N*sp['a'])
ax.set_yticks(np.arange(ylim[0], ylim[1], N*sp['a']));
# ax.set_ylim(-100)


# Calculate bandstructure

In [ ]:
formulas.fermi_wavenumber(params['mu'], 0.02) * syst_pars['a']

In [ ]:
def f(k):
    p = params.copy()
    p['k_x'] = k
    e, _ = spectrum.calc_spectrum(syst_straight, p, k= 8)
#     ham = syst_straight.hamiltonian_submatrix(params=p)
#     e = np.linalg.eigvalsh(ham)
    return np.sort(e)

In [ ]:
kf = formulas.fermi_wavenumber(params['mu'], 0.02) * syst_pars['a']
learner = adaptive.Learner1D(f, [-kf, kf])
# learner = adaptive.Learner1D(f, [0, .4])

In [ ]:
runner = adaptive.Runner(learner, ntasks=12)
runner.live_info()

In [ ]:
import formulas
formulas.fermi_wavenumber(params['mu'], 0.02) * syst_pars['a'] 

In [ ]:
kp
# 0.7449861537422936

In [ ]:
# %%output filename='z_y140nm'
learner.plot().redim(x='k_x', y='E(meV)')[:,-.2:.2]

In [ ]:
points = []
for k,v in learner.data.items():
    for e in v:
        points.append((k, e))

In [ ]:
hv.Scatter(points)
# [0:.4,-4:4]